<a href="https://colab.research.google.com/github/maxaubel/Scientific-Computing/blob/master/Tarea_1_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [4]:
# !pip install bitstring # to run on google colab
import bitstring as bs
import numpy as np

     |████████████████████████████████| 204kB 9.3MB/s 
  Created wheel for bitstring: filename=bitstring-3.1.7-cp37-none-any.whl size=37949 sha256=6b6141ec6e001b08d6a9fe9aca6cc34d32189ffc8bb948893acbdf57b5015dd1
  Stored in directory: /root/.cache/pip/wheels/b8/27/f0/8373e26b7de57db03dc18aaaebdd8c26a99da882416f762979
Successfully built bitstring


In [39]:
"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x, bits_mant, bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    
    bin_num = bs.pack('>d', x).bin
    sign_og, exp_og, mant_og = bin_num[0], bin_num[1:12], bin_num[12:12+bits_mant]
    sign_dec = 1
    if (sign_og == "1"): sign_dec = -1

    # Check first for special cases
    if exp_og == "1"*len(exp_og): 
        if mant_og == "0"*len(mant_og):
            return np.inf*sign_dec # Infinity and beyond
        else: return np.nan # Not a Number
    if bin_num == "0"*len(bin_num): return 0 # Zero
    
    # Mantissa
    mant_dec = 0
    for i, n in enumerate(mant_og):
        if(n == "1"): mant_dec += 2**-(i+1)
    
    # Exponent
    offset_og = int("1"*10, 2)
    dec_og = int(exp_og, 2) - offset_og
    if dec_og > 2**(bits_exp-1):
        dec_og = 2**(bits_exp-1)

    elif dec_og < -2**(bits_exp-1):
        return 0

    # Finally, "carefully" calculate the decimal
    num = sign_dec * (1+mant_dec) * (2**dec_og)

    return num

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado

    n1, n2 = f_new_rep(n1, bits_mant, bits_exp), f_new_rep(n2, bits_mant, bits_exp)
    result = f_new_rep(n1 + n2, bits_mant, bits_exp)
   
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado

    n1, n2 = f_new_rep(n1, bits_mant, bits_exp), f_new_rep(n2, bits_mant, bits_exp)
    result = f_new_rep(n1 * n2, bits_mant, bits_exp)
   
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado

    n1, n2 = f_new_rep(n1, bits_mant, bits_exp), f_new_rep(n2, bits_mant, bits_exp)
    result = f_new_rep(n1 / n2, bits_mant, bits_exp)
   
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp, with_sqrt=True):
    # Algoritmo para encontrar raíces de f(x).
    
    r_1 = 1 # Trivial root

    square = lambda x: f_multiplicacion(x, x, bits_mant, bits_exp)
    
    n = f_new_rep(square(A)-1, bits_mant, bits_exp)

    if not with_sqrt:
        non_trivial_root = f_new_rep(f_bisection(n, 1, n, square, bits_mant, bits_exp, depth=500), bits_mant, bits_exp)
    else:
        non_trivial_root = f_new_rep(np.sqrt(n), bits_mant, bits_exp)

    fraction = f_division(non_trivial_root, abs(A), bits_mant, bits_exp) 

    r_0 = 1 - fraction
    r_2 = 1 + fraction

    return (r_0, r_1, r_2)


"""
input
n         : (double) numero al que se le buscará la raíz
a         : (double) cota inferior de la biseccion
b         : (double) cota superior de la biseccion
f         : (function) funcion para la que se buscará la raiz
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (double) raiz
"""
def f_bisection(n, a, b, f, bits_mant, bits_exp, depth):
# Se implementó este método para calcular la raiz antes de que se dijera que se podía usar sqrt

    c = f_division( f_suma(a, b, bits_mant, bits_exp), 2, bits_mant, bits_exp);
    f_c = f(c); # c^2
    depth -= 1

    if ((f_c == n) or (depth <= 0)):
        return c
 
    elif (f_c < n):
        return f_bisection(n, c, b, f, bits_mant, bits_exp, depth)

    else:
        return f_bisection(n, a, c, f, bits_mant, bits_exp, depth)

In [45]:
["%.55f" % root for root in f_find_roots(A=3,bits_mant=5,bits_exp=5)]

['0.0625000000000000000000000000000000000000000000000000000',
 '1.0000000000000000000000000000000000000000000000000000000',
 '1.9375000000000000000000000000000000000000000000000000000']

### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


In [37]:
"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""

# Code based on Newton's method approach in https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1v2/03_roots_of_1D_equations.ipynb
# Modified to work on both sides of the requested x value
def raiz_de_oreman(alpha, beta, tol=1e-10):
    
    fp = lambda x: beta*np.cos(np.log(x))/x - alpha*np.sin(x)
    f  = lambda x: alpha*np.cos(x) + beta*np.sin(np.log(x))

    x0_low, x0_up = 23, 25.25

    hybrid_error_low = 100
    hybrid_error_up  = 100
    error_i_low = np.inf
    error_i_up  = np.inf
    max_iters = 100
    i = 1

    while (((hybrid_error_low > tol and hybrid_error_low < 1e12) or  
            (hybrid_error_up  > tol and hybrid_error_up  < 1e12)) and i<=max_iters):
        
        x1_low = x0_low - f(x0_low) / fp(x0_low)
        x1_up  = x0_up  - f(x0_up ) / fp(x0_up )

        if f(x1_low) == 0.0 or f(x1_up) == 0.0:
            hybrid_error = 0.0
            break
        
        hybrid_error_low = abs(x1_low-x0_low) / np.max([abs(x1_low), 1e-12])
        hybrid_error_up  = abs(x1_up-x0_up)   / np.max([abs(x1_up),  1e-12])
        
        error_iminus1_low = error_i_low
        error_iminus1_up  = error_i_up
        
        error_i_low = abs(x1_low-x0_low)
        error_i_up = abs(x1_up-x0_up)

        x0_low = x1_low
        x0_up = x1_up

        i+=1
        
        if (hybrid_error_low < tol) and (abs(x1_low-25) <= abs(x1_up-25)):
            return x1_low

        if (hybrid_error_up < tol) and (abs(x1_up-25) <= abs(x1_low-25)):
            return x1_up

    if i>=max_iters:
        print('Newtons Method did not converge. Too many iterations!!')
        return None
    else:
        print('Newtons Method did not converge!!')
        print(alpha, beta)
        return None

In [38]:
raiz_de_oreman(1.6, 20)

25.065629471024188

# Referencias

https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1v2/03_roots_of_1D_equations.ipynb